# Project Firefox MK I

In [ ]:
#calling library
from kinase_declarative import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
#create engine and bine the engine
engine = create_engine("sqlite:///kinase_database.db")
Base.metadata.bind = engine

In [ ]:
#create a session object
session = sessionmaker(bind=engine)
s = session()

## Get Uniprot gene_name and alias related to a kinase of query

In [ ]:
#search for things related to MAPK1
kinase = "MAPK1"
kinase_query = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
kinase_query.to_dict()

In [ ]:
def get_uniprot_gene_name(kinase):
    """
    Returns the preferred uniprot gene name.
    Returns a dictionary.
    """
    kinase_match = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
    return dict(kinase_match.to_dict())
    

In [60]:
#testing the functions
kinase = "MAPK1"
print(get_uniprot_gene_name(kinase))

{'gene_name': 'MAPK1', 'gene_alias': 'MAPK1'}


In [ ]:
kinase="PRKM1"
print(get_uniprot_gene_name(kinase))

In [ ]:
#test error
kinase = "error"
print(get_uniprot_gene_name(kinase))
#this will return error when run

## Get meta data for a gene name:

In [49]:
#get the meta data for a kinase whose name is the preferred name of uniprot
kinase = 'MAPK1'
kinase_query = s.query(KinaseGeneMeta).filter(KinaseGeneMeta.gene_name==kinase).one()
kinase_query.to_dict()

{'protein_name': 'Mitogen-activated protein kinase 1',
 'uniprot_number': 'P28482',
 'uniprot_entry': 'MK01_HUMAN',
 'gene_name': 'MAPK1',
 'kinase_family': 'CMGC Ser/Thr protein kinase family'}

In [50]:
#getting the meta data of a kinase who name is an alias
kinase = "PRKM1"
kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
kinase_query.to_dict()

{'protein_name': 'Mitogen-activated protein kinase 1',
 'uniprot_number': 'P28482',
 'uniprot_entry': 'MK01_HUMAN',
 'gene_name': 'MAPK1',
 'kinase_family': 'CMGC Ser/Thr protein kinase family'}

In [51]:
#getting the meta data of the kinase gene regardless of whether the name is the prefered gene name in uniprot or not
def get_meta_data(kinase):
    """
    Return the metadata of a gene based on its kinase gene name.
    It can be the prefered gene name or the alias/ synonyms.
    Returns an error if no gene was found.
    Returns a dictionary
    """
    kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
    return kinase_query.to_dict()

In [52]:
kinase = "KIAA0641"
print(get_meta_data(kinase))

{'protein_name': 'Serine/threonine-protein kinase LMTK1', 'uniprot_number': 'Q6ZMQ8', 'uniprot_entry': 'LMTK1_HUMAN', 'gene_name': 'AATK', 'kinase_family': 'Tyr protein kinase family'}


## Get subcellular location for a gene

In [59]:
kinase = "ALPK1"
kinase_query = s.query(KinaseSubcellularLocation).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
kinase_query[-1].to_dict()
for row in kinase_query:
    print(row.subcellular_location)

Cytosol
Cytoplasm


In [71]:
def get_subcellular_location(kinase):
    """
    Returns a list of subcellular location as an object for a kinase.
    """
    results = []
    kinase_query = s.query(KinaseSubcellularLocation).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
    for row in kinase_query:
        results.append(row.to_dict())
    return results
kinase = "ALPK1"
print(get_subcellular_location(kinase))

[{'subcellular_location_id': 129, 'gene_name': 'ALPK1', 'subcellular_location': 'Cytosol'}, {'subcellular_location_id': 130, 'gene_name': 'ALPK1', 'subcellular_location': 'Cytoplasm'}]


## Get inhibitor for a gene

In [67]:
#print out each object as a dictionary
kinase = "SGK1"
kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
for row in kinase_query:
    print(row.to_dict())

{'inhibitor_id': 1, 'inhibitor': 'GSK650394A', 'antagonizes_gene': 'SGK1', 'molecular_weight': 382.45, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/gsk-50394_5.jpg', 'empirical_formula': 'C25H22N2O2'}
{'inhibitor_id': 2, 'inhibitor': 'SGK-Sanofi-14i', 'antagonizes_gene': 'SGK1', 'molecular_weight': 430.87, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14i.jpg', 'empirical_formula': 'C18H15ClN6O3S'}
{'inhibitor_id': 3, 'inhibitor': 'SGK1-Sanofi-14g', 'antagonizes_gene': 'SGK1', 'molecular_weight': 435.29, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi--14g.jpg', 'empirical_formula': 'C17H12Cl2N6O2S'}
{'inhibitor_id': 4, 'inhibitor': 'SGK1-Sanofi-14h', 'antagonizes_gene': 'SGK1', 'molecular_weight': 435.29, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14h_0.jpg', 'empirical_formula': 'C17H12Cl2N6O2S'}
{'inhibitor_id': 5, 'inhibit

In [68]:
#print out each object's inhibitor
kinase = "SGK1"
kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
for row in kinase_query:
    print(row.inhibitor)

GSK650394A
SGK-Sanofi-14i
SGK1-Sanofi-14g
SGK1-Sanofi-14h
SGK1-Sanofi-14n


In [73]:
def get_inhibitor(kinase):
    """
    Return a list of dictionaries.
    """
    results = []
    kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
    for row in kinase_query:
        results.append(row.to_dict())
    return results
kinase = "MARK1"
get_inhibitor(kinase)

[{'inhibitor_id': 94,
  'inhibitor': 'MRT199665',
  'antagonizes_gene': 'MARK1',
  'molecular_weight': 506.04,
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/mrt-199665.jpg',
  'empirical_formula': 'C28H31N5O2.ClH'}]

## Get substrates meta data

In [74]:
kinase = "MAPK1"
kinase_query = s.query(SubstrateMeta).join(KinaseGeneName, ).join(KinaseSubstrateRelations).join(PhosphositeMeta).filter(KinaseGeneName.gene_alias==kinase).all()

InvalidRequestError: Don't know how to join to <class 'kinase_declarative.KinaseGeneName'>; please use an ON clause to more clearly establish the left side of this join